# GoogleNet

# Inception块

Inception块里有4条并行的线路。  
前3条线路使用窗口大小分别是1×1、3×3和5×5的卷积层来抽取不同空间尺寸下的信息，其中中间2个线路会对输入先做1×11×1卷积来减少输入通道数，以降低模型复杂度。  
第四条线路则使用3×3最大池化层，后接1×1卷积层来改变通道数。  
4条线路都使用了合适的填充来使输入与输出的高和宽一致。最后我们将每条线路的输出在通道维上连结，并输入接下来的层中去。

In [11]:
import torch
from torch import nn,optim

class Inception(nn.Module):
    def __init__(self,inc,c1,c2,c3,c4):
        super().__init__()
        #线路1 单1 x 1卷积层
        self.p1=nn.Sequential(
            nn.Conv2d(inc,c1,1),
            nn.ReLU()
        )
        #线路2 1 x 1卷积层后接3 x 3卷积层
        self.p2=nn.Sequential(
             nn.Conv2d(inc, c2[0], 1),
             nn.ReLU(),
             nn.Conv2d(c2[0],c2[1], 3, padding=1),
             nn.ReLU()
        )
        #线路3 1 x 1卷积层后接5 x 5卷积层
        self.p3=nn.Sequential(
            nn.Conv2d(inc, c3[0], 1),
            nn.ReLU(),
            nn.Conv2d(c3[0],c3[1],5,padding=2),
            nn.ReLU()
        )
        #线路4 3 x 3最大池化层后接1 x 1卷积层
        self.p4=nn.Sequential(
            nn.MaxPool2d(3,1,1),
            nn.ReLU(),
            nn.Conv2d(inc,c4,1),
            nn.ReLU()
        )
    def forward(self,input):
        p1=self.p1(input)
        p2=self.p2(input)
        p3=self.p3(input)
        p4=self.p4(input)

        return torch.cat((p1,p2,p3,p4),dim=1)


# GoogleNet模型

GooglNet跟VGG一样，在主体卷积部分中使用5个模块（block），每个模块之间使用歩幅为2的3*3的最大池化层来减小输出高宽

第一个模块使用64通道的7*7卷积层

In [12]:
b1=nn.Sequential(
    nn.Conv2d(1,64,7,2,3),
    nn.ReLU(),
    nn.MaxPool2d(3,3,1)
)

第二个模块使用2个卷积层：首先是64通道的1*1卷积层  
然后将通道数增大3倍的3*3卷积层

In [13]:
b2=nn.Sequential(
    nn.Conv2d(64,64,1),
    nn.Conv2d(64,192,3,padding=1),
    nn.MaxPool2d(3,2,1)
)

第三模块串联2个完整的Inception块。  
第一个Inception块的输出通道数为64+128+32+32=25664+128+32+32=256，其中4条线路的输出通道数比例为64:128:32:32=2:4:1:164:128:32:32=2:4:1:1。其中第二、第三条线路先分别将输入通道数减小至96/192=1/296/192=1/2和16/192=1/1216/192=1/12后，再接上第二层卷积层。  
第二个Inception块输出通道数增至128+192+96+64=480128+192+96+64=480，每条线路的输出通道数之比为128:192:96:64=4:6:3:2128:192:96:64=4:6:3:2。其中第二、第三条线路先分别将输入通道数减小至128/256=1/2128/256=1/2和32/256=1/832/256=1/8。

In [14]:
b3 = nn.Sequential(Inception(192, 64, (96, 128), (16, 32), 32),
                   Inception(256, 128, (128, 192), (32, 96), 64),
                   nn.MaxPool2d(kernel_size=3, stride=2, padding=1))

第四模块更加复杂。  
它串联了5个Inception块，其输出通道数分别是192+208+48+64=512192+208+48+64=512、160+224+64+64=512160+224+64+64=512、128+256+64+64=512128+256+64+64=512、112+288+64+64=528112+288+64+64=528和256+320+128+128=832256+320+128+128=832。这些线路的通道数分配和第三模块中的类似，首先含3×3卷积层的第二条线路输出最多通道，其次是仅含1×1卷积层的第一条线路，之后是含5×5卷积层的第三条线路和含3×3最大池化层的第四条线路。其中第二、第三条线路都会先按比例减小通道数。这些比例在各个Inception块中都略有不同。

In [15]:
b4 = nn.Sequential(Inception(480, 192, (96, 208), (16, 48), 64),
                   Inception(512, 160, (112, 224), (24, 64), 64),
                   Inception(512, 128, (128, 256), (24, 64), 64),
                   Inception(512, 112, (144, 288), (32, 64), 64),
                   Inception(528, 256, (160, 320), (32, 128), 128),
                   nn.MaxPool2d(kernel_size=3, stride=2, padding=1))

第五模块有输出通道数为256+320+128+128=832256+320+128+128=832和384+384+128+128=1024384+384+128+128=1024的两个Inception块。其中每条线路的通道数的分配思路和第三、第四模块中的一致，只是在具体数值上有所不同。需要注意的是，第五模块的后面紧跟输出层，该模块同NiN一样使用全局平均池化层来将每个通道的高和宽变成1。最后我们将输出变成二维数组后接上一个输出个数为标签类别数的全连接层。

In [17]:
class GlobalAvgPool2d(nn.Module):
    def __init__(self):
        super().__init__()
    def forward(self,x):
        return torch.nn.functional.adaptive_avg_pool2d(x, (1,1))

b5 = nn.Sequential(Inception(832, 256, (160, 320), (32, 128), 128),
                   Inception(832, 384, (192, 384), (48, 128), 128),
                   GlobalAvgPool2d())

net = nn.Sequential(b1, b2, b3, b4, b5, 
                    nn.Flatten(), nn.Linear(1024, 10))

In [19]:
x = torch.rand(1, 1, 96, 96)
for name,blk in net.named_children():
    x=blk(x)
    print(name,x.shape)

0 torch.Size([1, 64, 16, 16])
1 torch.Size([1, 192, 8, 8])
2 torch.Size([1, 480, 4, 4])
3 torch.Size([1, 832, 2, 2])
4 torch.Size([1, 1024, 1, 1])
5 torch.Size([1, 1024])
6 torch.Size([1, 10])


# 获取数据集

In [21]:
import torchvision

transform = torchvision.transforms.Compose(
    [torchvision.transforms.Resize(size=96),
    torchvision.transforms.ToTensor()]
)

#获取数据集
mnist_train = torchvision.datasets.FashionMNIST(root='~/Datasets/FashionMNIST', train=True, download=True, transform=transform)
mnist_test = torchvision.datasets.FashionMNIST(root='~/Datasets/FashionMNIST', train=False, download=True, transform=transform)
#读取数据集
batchSize=128
trainIter=torch.utils.data.DataLoader(mnist_train,batch_size=batchSize,shuffle=True,num_workers=8)
testIter=torch.utils.data.DataLoader(mnist_test,batch_size=batchSize,shuffle=True,num_workers=8)


# 评价

In [22]:
def evaluate_accuracy(data_iter, net):
    device=torch.device('cuda')
    acc_sum, n = 0.0, 0
    with torch.no_grad():
        net.eval() # 评估模式
        for X, y in data_iter:
            acc_sum += (net(X.to(device)).argmax(dim=1) == y.to(device)).float().sum().item()
            n += y.shape[0]
        net.train() # 改回训练模式 
    return acc_sum / n

# 训练

In [23]:
lr,epochsNum = 0.001,5
net=net.cuda()
loss=torch.nn.CrossEntropyLoss()
optimzer=torch.optim.Adam(net.parameters(),lr)

for epoch in range(epochsNum):
    train_l_sum=n=train_acc_sum=0
    for x,y in trainIter:
        x=x.cuda()
        y=y.cuda()
        yP=net(x)
        l=loss(yP,y)
        l.backward() 
        optimzer.step()
        optimzer.zero_grad()
        train_l_sum += l.item()
        train_acc_sum += (yP.argmax(dim=1) == y).sum().item()
        n +=y.shape[0]
    print(epoch,train_l_sum/n,train_acc_sum/n,evaluate_accuracy(testIter,net))

0 0.008870575345059237 0.5308166666666667 0.8011
1 0.0034264414305488267 0.8415166666666667 0.8573
2 0.0027164866690834364 0.87275 0.8683
3 0.0023870243221521376 0.8875666666666666 0.8899
4 0.0020869226388633253 0.9016833333333333 0.8904
